# infoGAN

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
a,b  = mnist.train.next_batch(1)
print(a.shape)
print(b.shape)
print(b)

(1, 784)
(1, 10)
[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


# utility func

In [3]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics_gpu1"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "d_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["q_loss"], label="q loss", color="g")
    plt.legend()
    plt.savefig(os.path.join(path, "q_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.plot(metrics["q_loss"], label="q loss", color="g")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [4]:
# plot images
def save_imgs(images, plot_dim=(10,6), size=(6,10), epoch=None):
    # make directory if there is not
    path = "generated_figures_gpu1"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((28, 28))
        plt.tight_layout()
        plt.imshow(img, cmap="gray")
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [5]:
# training
import pickle
import numpy as np
import os

def unpickle(file):
    fo = open(file, 'rb')
    #print(file)
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def one_hot_vec(label):
    vec = np.zeros(10)
    vec[label] = 1
    return vec

def load_data():
    x_all = []
    y_all = []
    for i in range (5):
        d = unpickle("cifar-10-batches-py/data_batch_" + str(i+1))
        x_ = d['data']
        y_ = d['labels']
        x_all.append(x_)
        y_all.append(y_)

    d = unpickle('cifar-10-batches-py/test_batch')
    x_all.append(d['data'])
    y_all.append(d['labels'])

    x = -0.5 + (np.concatenate(x_all) / np.float32(255))
    y = np.concatenate(y_all)
    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3))

    #pixel_mean = np.mean(x[0:50000],axis=0)
    #x -= pixel_mean
    y = np.array(list(map(one_hot_vec, y)))
    X_train = x[0:50000,:,:,:]
    Y_train = y[0:50000]
    #X_test = x[50000:,:,:,:]
    #Y_test = y[50000:]

    #return (X_train, Y_train, X_test, Y_test)
    return X_train, Y_train

# model

In [6]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 28*28*1 
        self.z_dim = 100


    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            inputs = z
            
            fc1 = tf.layers.dense(inputs, 7*7*512, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # noise???
            fc1 = tf.nn.relu(fc1)
            """
            fc2 = tf.layers.dense(fc1, 128*7*7, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # noise???
            #fc2 = tf.layers.batch_normalization(fc1)
            #fc1 = tf.nn.relu(fc1)
            """
            fc1 = tf.reshape(fc1, [-1, 7, 7, 512])

            conv1 = tf.layers.conv2d_transpose(fc1, 256, [4,4],[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14
            #conv1 = tf.layers.batch_normalization(conv1)
            conv1 = tf.nn.relu(conv1)  
            #conv1 = tf.nn.tanh(conv1)

            conv2 = tf.layers.conv2d_transpose(conv1, 1, [4,4],[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 28x28
            #conv2 = tf.layers.batch_normalization(conv2)

            #conv2 = tf.nn.tanh(conv2)
            #conv2 = tf.nn.sigmoid(conv2)
            conv2 = tf.reshape(conv2, [-1, 28*28*1])
            print(conv2.get_shape())
            outputs = conv2
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [7]:
class Discriminator:
    def __init__(self, cat_size, con_size):
        self.reuse = False
        self.X_dim = 28*28*1
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.cat_size = cat_size
        self.con_size = con_size        
        
    def __call__(self, x,training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 28, 28, 1])

            conv1 = tf.layers.conv2d(x, 128, [4,4], [2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14x64
            conv1 = leaky_relu(conv1)

            #print(conv1.get_shape())

            #conv1 = tf.concat([conv1, y], 3)

            conv2 = tf.layers.conv2d(conv1, 256, [4,4], [2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14
            conv2 = tf.layers.batch_normalization(conv2)
            conv2 = leaky_relu(conv2)
            conv2 = tf.contrib.layers.flatten(conv2)
            
            fc1 = tf.layers.dense(conv2, 512, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            fc1 = leaky_relu(fc1)

            fc2 = tf.layers.dense(fc1, 128, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            fc2 = leaky_relu(fc2)
            
            disc = tf.layers.dense(fc2, 1,  kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            disc = tf.squeeze(disc, -1)
            print(disc.get_shape())
            
        with tf.name_scope('q' + name), tf.variable_scope('q', reuse=self.reuse):               
            cat = tf.layers.dense(fc2, self.cat_size, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            con = tf.layers.dense(fc2, self.con_size, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            con = tf.nn.tanh(con)

        self.reuse = True
        self.d_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        self.q_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='q')
        return disc, cat, con

In [8]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [9]:
def one_hot_vec(label):
    vec = np.zeros(10)
    vec[label] = 1
    return vec

In [ ]:
class GAN:
    def __init__(self):
        self.batch_size = 256
        self.img_size = 28
        self.cat_size = 10
        self.con_size = 2
        self.rand_size = 88

        self.epochs = 100000
        self.epoch_saveMetrics = 1000
        self.epoch_saveSampleImg = 1000
        self.epoch_saveParamter = 10000
        self.losses = {"d_loss":[], "g_loss":[], "q_loss":[]}

        #self.x, self.y = load_data()

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 1])
        #self.Y_tr = tf.placeholder(tf.float32, shape=[None, num_class])
        self.cat_label = tf.placeholder(tf.int32, [None])
        self.cat = tf.placeholder(tf.float32, [None, self.cat_size])
        self.con = tf.placeholder(tf.float32, [None, self.con_size])
        self.Z1 = tf.placeholder(tf.float32, [None, self.rand_size+self.cat_size+self.con_size])
        
        self.g = Generator()
        self.d = Discriminator(self.cat_size, self.con_size)
        self.Xg = self.g(self.Z1)

    def loss(self):
        disc_tr, cat_tr, con_tr = self.d(self.X_tr)
        disc_gen, cat_gen, con_gen = self.d(self.Xg)
        
        print(disc_tr.get_shape())
        #loss_d_tr = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_tr, labels=tf.ones_like(disc_tr)))
        #loss_d_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.zeros_like(disc_gen)))
        #loss_d = (loss_d_tr + loss_d_gen)/2

        #loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.ones_like(disc_gen)))

        loss_d = -tf.reduce_mean(tf.log(disc_tr) + tf.log(1-disc_gen))
        loss_g = -tf.reduce_mean(tf.log((disc_gen/(1-disc_gen))))

        # categorical factorloss
        loss_cat = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=cat_gen, labels=self.cat_label))
        
        # continuous factor loss
        loss_con = tf.reduce_sum(0.5*tf.square(con_gen - self.con), axis=1)
        loss_con = tf.reduce_mean(loss_con)
        #loss_con = tf.reduce_mean(tf.square(con_gen - self.con))

        d_cost = loss_d# + loss_cat + loss_con
        g_cost = loss_g# + loss_cat + loss_con
        q_cost = loss_con + loss_cat

        return g_cost, d_cost, q_cost

    def train(self):
        # Optimizer
        d_lr = 2e-4
        g_lr = 2e-3
        q_lr = 2e-4

        self.L_g, self.L_d, self.L_q = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.d_variables)

        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        q_opt = tf.train.AdamOptimizer(learning_rate=q_lr)
        q_train_op = q_opt.minimize(self.L_q, var_list=self.d.q_variables)

        saver = tf.train.Saver()
        #%debug

        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list="1"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())
            epoch_pre = 0
            path = "model_infoGAN"
            #saver.restore(sess, path+"/dcgan_model" + str(epoch_pre) + ".ckpt")
            bs = 60
            test_z_rand = np.random.uniform(-1, 1, size=[bs, self.rand_size])
            test_z_cat_label = np.repeat(np.arange(10), bs/10)
            test_z_cat = np.array(list(map(one_hot_vec, test_z_cat_label)))

            test_z_con = np.random.uniform(-1, 1, size=[bs, self.con_size])
            print(test_z_rand.shape)
            print(test_z_cat_label.shape)
            print(test_z_cat.shape)
            print(test_z_con.shape)
            #from IPython.core.debugger import Pdb; Pdb().set_trace()
            test_z = np.concatenate((test_z_rand,test_z_cat,test_z_con), axis=1)

            for epoch in range(self.epochs):              
                for _ in range(1):
                    # 訓練データを抜粋
                    X_mb, Y_mb = mnist.train.next_batch(self.batch_size)
                    X_mb = np.reshape(X_mb, [-1, 28, 28, 1])

                    z_rand = np.random.uniform(-1, 1, size=[self.batch_size, self.rand_size])
                    z_cat_label = np.random.randint(0, 10, [self.batch_size])
                    #z_cat_label = np.argmax(Y_mb, axis=1)
                    z_cat = np.array(list(map(one_hot_vec, z_cat_label)))

                    z_con = np.random.uniform(-1, 1, size=[self.batch_size, self.con_size])
                    z = np.concatenate((z_rand,z_cat,z_con), axis=1)
                    #from IPython.core.debugger import Pdb; Pdb().set_trace()
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                        self.X_tr: X_mb,
                        self.Z1:z,
                        self.cat_label:z_cat_label,
                        #self.cat: z_cat,
                        self.con: z_con,
                    })

                # train G
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                        self.X_tr: X_mb,
                        self.Z1:z,
                        self.cat_label:z_cat_label,
                        #self.cat: z_cat,
                        self.con: z_con,
                })

                _, q_loss_value = sess.run([q_train_op, self.L_q], feed_dict={
                        self.X_tr: X_mb,
                        self.Z1:z,
                        self.cat_label:z_cat_label,
                        #self.cat: z_cat,
                        self.con: z_con,
                })

                # generate Sample Imgs
                #sampleImgsOfX2Y, sampleImgsOfY2X = sess.run([self.X2Y, self.Y2X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # 結果をappend
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                self.losses["q_loss"].append(np.sum(q_loss_value))

                if epoch % 100 == 0:
                    print("epoch:" + str(epoch+epoch_pre))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 1:
                    save_metrics(self.losses, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.Z1: test_z})
                    #img = img
                    #img += 0.5
                    #img *= 255
                    #print(img.shape)
                    save_imgs(img, epoch=str(epoch+epoch_pre))
                # parameterのsave
                if epoch % self.epoch_saveParamter == 1:
                    path = "model_gpu1"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, path+"/dcgan_model" + str(epoch+epoch_pre) + ".ckpt")
       

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = GAN()
gan.train()

(?, 784)
(?,)
(?,)
(?,)


/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:79: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


(60, 88)
(60,)
(60, 10)
(60, 2)
epoch:0
epoch:100
epoch:200
epoch:300
epoch:400
epoch:500
epoch:600
epoch:700
epoch:800
epoch:900
epoch:1000
epoch:1100
epoch:1200
epoch:1300
epoch:1400
epoch:1500
epoch:1600
epoch:1700
epoch:1800
epoch:1900
epoch:2000
epoch:2100
epoch:2200
epoch:2300
epoch:2400
epoch:2500
epoch:2600
epoch:2700
epoch:2800
epoch:2900
epoch:3000
epoch:3100
epoch:3200
epoch:3300
epoch:3400
epoch:3500
epoch:3600
epoch:3700
epoch:3800
epoch:3900
epoch:4000
epoch:4100
epoch:4200
epoch:4300
epoch:4400
epoch:4500
epoch:4600
epoch:4700
epoch:4800
epoch:4900
epoch:5000
epoch:5100
